In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import random
import os
from sklearn.metrics import accuracy_score

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import EfficientNetB2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.applications.efficientnet import preprocess_input

In [3]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [4]:

SEED = 42
tf.random.set_seed(SEED)
np.random.seed(SEED)
import os

os.environ['PYTHONHASHSEED'] = str(SEED)
os.environ['TF_DETERMINISTIC_OPS'] = '1'

In [5]:

BATCH_SIZE = 32
IMG_SIZE = 288
TRAIN_PATH = '/content/drive/MyDrive/FaceShape Dataset/training_set'
TEST_PATH ='/content/drive/MyDrive/FaceShape Dataset/testing_set'


In [6]:
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    horizontal_flip=True,
    rotation_range=10,
    brightness_range=[0.8, 1.2],
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    validation_split=0.0
)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

In [7]:
train_generator = train_datagen.flow_from_directory(
    TRAIN_PATH,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True
)

test_generator = test_datagen.flow_from_directory(
    TEST_PATH,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

Found 4007 images belonging to 5 classes.
Found 1000 images belonging to 5 classes.


In [8]:

num_classes = train_generator.num_classes
class_indices = train_generator.class_indices
print(f"Classes: {class_indices}")
print(f"Number of classes: {num_classes}")

Classes: {'Heart': 0, 'Oblong': 1, 'Oval': 2, 'Round': 3, 'Square': 4}
Number of classes: 5


In [9]:


def create_model(num_classes):
    base_model = EfficientNetB2(
        weights='imagenet',
        include_top=False,
        pooling='avg',
        input_shape=(IMG_SIZE, IMG_SIZE, 3)
    )


    base_model.trainable = True


    model = keras.Sequential([
        base_model,
        #layers.GlobalAveragePooling2D(),
        layers.Dropout(0.2),
        layers.Dense(num_classes, activation='softmax')
    ])

    return model

model = create_model(num_classes)

31790344/31790344 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [10]:

model.compile(
    optimizer=keras.optimizers.AdamW(
        learning_rate=5e-3,
        beta_1=0.9,
        beta_2=0.999,
        weight_decay=0.01
    ),
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing = 0.1),
    metrics=['accuracy']
)


model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ efficientnetb2 (Functional)     │ (None, 1408)           │     7,768,569 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1408)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 5)              │         7,045 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,775,614 (29.66 MB)

 Trainable params: 7,708,039 (29.40 MB)

 Non-trainable params: 67,575 (263.97 KB)

In [11]:
# Quantization
def representative_data_gen():
    sample_count = 0
    max_samples = 100

    for batch_images, _ in train_generator:
        for image in batch_images:
            if sample_count >= max_samples:
                return

            yield [np.expand_dims(image, axis=0).astype(np.float32)]
            sample_count += 1
        if sample_count >= max_samples:
            break

In [12]:
def convert_to_tflite_int8(model, model_name="model_int8.tflite"):

    converter = tf.lite.TFLiteConverter.from_keras_model(model)


    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    converter.representative_dataset = representative_data_gen
    converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
    converter.inference_input_type = tf.int8
    converter.inference_output_type = tf.int8


    quantized_tflite_model = converter.convert()


    with open(model_name, 'wb') as f:
        f.write(quantized_tflite_model)




    original_size = os.path.getsize('best_model.h5') / (1024 * 1024)  # MB
    quantized_size = os.path.getsize(model_name) / (1024 * 1024)  # MB

    print(f"model size: {original_size:.2f} MB")
    print(f"quantized model size: {quantized_size:.2f} MB")

    return quantized_tflite_model
model.load_weights('best_model.h5')
quantized_model = convert_to_tflite_int8(model)

Saved artifact at '/tmp/tmprscapmxf'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 288, 288, 3), dtype=tf.float32, name='keras_tensor_341')
Output Type:
  TensorSpec(shape=(None, 5), dtype=tf.float32, name=None)
Captures:
  134088181065552: TensorSpec(shape=(1, 1, 1, 3), dtype=tf.float32, name=None)
  134088181066704: TensorSpec(shape=(1, 1, 1, 3), dtype=tf.float32, name=None)
  134088186705872: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134088186705296: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134088186704144: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134088186706448: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134088186698000: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134088186706832: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134088184759312: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134088184758928: TensorSpec(shape=(), dtype=tf.resource, name=

/usr/local/lib/python3.11/dist-packages/tensorflow/lite/python/convert.py:997: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


model size: 89.39 MB
quantized model size: 8.71 MB


In [13]:
import time
def test_tflite_model(tflite_model_path, test_generator):


    interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
    interpreter.allocate_tensors()


    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    print(f"Input tensor shape: {input_details[0]['shape']}")
    print(f"Input tensor type: {input_details[0]['dtype']}")
    print(f"Output tensor shape: {output_details[0]['shape']}")
    print(f"Output tensor type: {output_details[0]['dtype']}")


    input_scale, input_zero_point = input_details[0]['quantization']
    output_scale, output_zero_point = output_details[0]['quantization']

    print(f"Input - Scale: {input_scale}, Zero point: {input_zero_point}")
    print(f"Output - Scale: {output_scale}, Zero point: {output_zero_point}")


    all_predictions = []
    all_true_labels = []
    inference_times = []

    batch_count = 0
    total_samples = 0


    test_generator.reset()

    for batch_images, batch_labels in test_generator:
        batch_count += 1

        for i in range(len(batch_images)):

            image = batch_images[i]
            # image was preprocessed.

            if input_details[0]['dtype'] == np.uint8:
                image = image / input_scale + input_zero_point
                image = np.clip(image, 0, 255).astype(np.uint8)

            elif input_details[0]['dtype'] == np.int8:
               image = image / input_scale + input_zero_point
               image = np.clip(image, -128, 127).astype(np.int8)

            image = np.expand_dims(image, axis=0)


            start_time = time.time()


            interpreter.set_tensor(input_details[0]['index'], image)


            interpreter.invoke()


            output_data = interpreter.get_tensor(output_details[0]['index'])

            end_time = time.time()
            inference_times.append((end_time - start_time) * 1000)


            if output_details[0]['dtype'] == np.uint8 or output_details[0]['dtype'] == np.int8:
                predictions = (output_data.astype(np.float32) - output_zero_point) * output_scale
            else:
                predictions = output_data


            predictions = predictions / np.sum(predictions, axis=-1, keepdims=True)

            all_predictions.append(np.argmax(predictions))
            all_true_labels.append(np.argmax(batch_labels[i]))
            total_samples += 1
        if batch_count >= len(test_generator):
            break


    accuracy = accuracy_score(all_true_labels, all_predictions)
    avg_inference_time = np.mean(inference_times)

    print(f"TFLite accuracy: {accuracy:.4f}")
    print(f"Inference time: {avg_inference_time:.2f} ms")


    return accuracy, avg_inference_time

test_tflite_model("model_int8.tflite",test_generator)

Input tensor shape: [  1 288 288   3]
Input tensor type: <class 'numpy.int8'>
Output tensor shape: [1 5]
Output tensor type: <class 'numpy.int8'>
Input - Scale: 1.0, Zero point: -128
Output - Scale: 0.00390625, Zero point: -128
TFLite accuracy: 0.7500
Inference time: 73.78 ms


(0.75, np.float64(73.78081011772156))

In [14]:
def test_original_model_inference(original_model, test_generator):

    test_generator.reset()

    inference_times = []
    all_predictions = []
    all_true_labels = []
    total_samples = 0
    batch_count = 0

    for batch_images, batch_labels in test_generator:
        batch_count += 1

        for i in range(len(batch_images)):
            image = np.expand_dims(batch_images[i], axis=0)


            start_time = time.time()
            prediction = original_model.predict(image, verbose=0)
            end_time = time.time()

            inference_times.append((end_time - start_time) * 1000)

            all_predictions.append(np.argmax(prediction))
            all_true_labels.append(np.argmax(batch_labels[i]))
            total_samples += 1
        if batch_count >= len(test_generator):
            break

    accuracy = accuracy_score(all_true_labels, all_predictions)
    avg_inference_time = np.mean(inference_times)



    print(f"model accuracy: {accuracy:.4f} ({accuracy*100:.2f}%)")
    print(f"inference time: {avg_inference_time:.2f} ms")


    return accuracy, avg_inference_time
test_original_model_inference(model,test_generator)

model accuracy: 0.8330 (83.30%)
inference time: 160.31 ms


(0.833, np.float64(160.31271696090698))